# Preprocessing Coin  
Combining features (Price and Volumes) and normalizing the values

In [ ]:
import pandas as pd
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
import pandas_datareader as web
import yfinance as yf
from yahoofinancials import YahooFinancials
import time
from datetime import datetime, timedelta, date, time
import os
import yfinance as yf
import datetime as dt

In [3]:
price_data = pd.read_csv('..\Data\CryptoClosePrices.csv')
price_data.head(5)

Obtain data of only one coin : Cardano (ADA), rename the column name to Close, to indicate closing price.

In [10]:
coin_price_data = coin_price_data["ADA-USD"]
coin_price_data.rename(columns={'ADA-USD':'close'}, inplace = True)
coin_price_data['previous_close'] = coin_price_data['close'].shift(1) 
coin_price_data.fillna(0)

Integrate sentiment data to price data

In [12]:
sentiment_score = pd.read_csv("../Data/Cardano.csv") 
coin_final_data = coin_price_data.join(sentiment_score['Score'])

In [ ]:
coin_final_data.dropna()
coin_final_data.sort_values(by="dates")
coin_final_data.rename(columns={'Score':'sentiment'}, inplace = True)
coin_final_data = coin_final_data[['dates','previous_close','close','sentiment']]
coin_final_data.head()

In [ ]:
# Replace "DesiredTicker" with the ticker symbol of the cryptocurrency you want to analyze
ticker = yf.Ticker("DesiredTicker") #ETH-USD, BTC-USD, etc, 

# Set the start and end date of the date range
d0 = date(2023, 1, 2)
d1 = date(2021, 1, 1)

# Get the historical market data for the date range
history = ticker.history(start=d1, end=d0)

# Extract the trading volume for each day
volume = history['Volume']
volume = pd.DataFrame(volume)
volume = volume.reset_index()

In [ ]:
def get_digits(num):
    count = 0
    while(num > 0):
        num = num//10
        count = count + 1
    return count

def normalize(coin): 
    max_num = coin.max()
    max_len_max_price = get_digits(max_num)
    return coin/10**max_len_max_price

#normalizing the volume
volume = volume['Volume']  
for i in range (0, len(volume)):
    volume[i] = normalize(volume[i])
coin_final_data['previous_volume'] = volume

#normalizing the prices
close = coin_final_data["close"].values
for i in range (0, len(close)):
    close[i] = normalize(close[i])
coin_final_data['close_eth'] = close

previous_close = coin_final_data["previous_close"].values
for i in range (0,len(previous_close)):
    previous_close[i] = normalize(previous_close[i])
coin_final_data['previous_close_eth'] = previous_close

In [167]:
coin_final_data = coin_final_data.drop(columns=['close','previous_close'])
coin_final_data.rename(columns={'previous_volume':'pre_volu_eth'}, inplace = True)
coin_final_data.to_csv('ethereum_final_training_data2.csv')